# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **economic activity or measures** (see below) for the region of **Ann Arbor, Michigan, United States**, or **United States** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Ann Arbor, Michigan, United States** to Ann Arbor, USA. In that case at least one source file must be about **Ann Arbor, Michigan, United States**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Ann Arbor, Michigan, United States** and **economic activity or measures**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **economic activity or measures**?  For this category you might look at the inputs or outputs to the given economy, or major changes in the economy compared to other regions.

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib notebook

# load countries database to map from iso3 country code to country name
df_countries = pd.read_csv('countries.csv')

# load and read only necessary columns from global terrorism database
df_gtd = pd.read_csv('gtd.csv', encoding='latin1', low_memory=False)

# load world religion database
df_wrd = pd.read_csv('wrd.csv', sep=';')

# all the countries iso3 code that we are interested in
iso3s = ['USA', 'GBR', 'CAN', 'DEU', 'CHN', 'PAK', 'IND', 'BGD']

# all the religions we are interested in
religions = ['christianity', 'islam', 'buddhism', 'hindu', 'jews']

# years of interest
years = [1970, 1975, 1980, 1985, 1990, 1995, 2000, 2005, 2010]


In [ ]:
# method to add the iso alpha-3 country codes into global terrorism database
def copy_country_codes(df):
    df_gtd.loc[df_gtd['country_txt'] == df['name'], 'alpha-3'] = df['alpha-3']
    return df


def calculate_attack_percentage(df):
    df['percentage'] = df['attack_count'] / total_attacks_by_year[df['iyear']] * 100
    return df

df_countries = df_countries[['name', 'alpha-3']]

# keep only the necessary columns, year and country_name
df_gtd = df_gtd[['iyear', 'country_txt']]
df_gtd['attack_count'] = 1

# treat East and West Germany as Germany
df_gtd.loc[df_gtd['country_txt'].str.contains('Germany'), 'country_txt'] = 'Germany'

# keep only those years which are in years
df_gtd = df_gtd[df_gtd['iyear'].isin(years)]

# add an empty column to put iso alpha-3 country codes
df_gtd['alpha-3'] = ''

# copy country codes from df_countries
_ = df_countries.apply(copy_country_codes, axis=1)


# keep only those countries data which are in iso3s
df_gtd = df_gtd[df_gtd['alpha-3'].isin(iso3s)]

# calculate the cumulative terror attack by year and country
# df_gtdc = pd.DataFrame(columns=['year', 'iso3', 'total_attack'])

df_gtdc = (df_gtd.groupby(['iyear', 'alpha-3']).size()).to_frame()
df_gtdc.reset_index(inplace=True)
df_gtdc.columns = ['iyear', 'iso3', 'attack_count']
df_gtdc['percentage'] = 0.0

total_attacks_by_year = df_gtdc.groupby(['iyear'])['attack_count'].sum()

df_gtdc = df_gtdc.apply(calculate_attack_percentage, axis=1)

In [ ]:
# clean up the wrd religion data
df_wrdc = pd.DataFrame()

# prepare the new cleaned up data frame df_wrdc
df_wrdc['year'] = df_wrd['YEAR']
df_wrdc['iso3'] = df_wrd['ISO3']
df_wrdc['country'] = df_wrd['COUNTRY']
df_wrdc['christianity'] = df_wrd['CHRSPROT'] + df_wrd['CHRSCAT'] + df_wrd['CHRSORTH'] + df_wrd['CHRSANG'] + df_wrd['CHRSOTHR'] + df_wrd['CHRSGEN']
df_wrdc['islam'] = df_wrd['ISLMSUN'] + df_wrd['ISLMSHI'] + df_wrd['ISLMIBD'] + df_wrd['ISLMNAT'] + df_wrd['ISLMALW'] + df_wrd['ISLMAHM'] + df_wrd['ISLMOTHR'] + df_wrd['ISLMGEN']
df_wrdc['buddhism'] = df_wrd['BUDMAH'] + df_wrd['BUDTHR'] + df_wrd['BUDOTHR'] + df_wrd['BUDGEN']
df_wrdc['jews'] = df_wrd['JUDORTH'] + df_wrd['JDCONS'] + df_wrd['JUDREF'] + df_wrd['JUDOTHR'] + df_wrd['JUDGEN']
df_wrdc['hindu'] = df_wrd['HINDGEN']


# drop any data rows not belongs to our countries of interest
df_wrdc = df_wrdc[df_wrdc['iso3'].isin(iso3s)]

# replace all 0 population count with mean value of that column
for religion in religions:
    religion_means = df_wrdc.groupby(['iso3'])[religion].mean()
    for (iso3, mean) in religion_means.iteritems():
        df_wrdc.loc[(df_wrdc[religion] == 0) & (df_wrdc['iso3'] == iso3), religion] = mean

# create new column total with total population
df_wrdc['total'] = df_wrdc['christianity'] + df_wrdc['islam'] + df_wrdc['buddhism'] + df_wrdc['jews'] + df_wrdc['hindu']
df_wrdc['christianity_p'] = df_wrdc['christianity'] / df_wrdc['total'] * 100
df_wrdc['islam_p'] = df_wrdc['islam'] / df_wrdc['total'] * 100
df_wrdc['buddhism_p'] = df_wrdc['buddhism'] / df_wrdc['total'] * 100
df_wrdc['jews_p'] = df_wrdc['jews'] / df_wrdc['total'] * 100
df_wrdc['hindu_p'] = df_wrdc['hindu'] / df_wrdc['total'] * 100

# drop any data rows before 1970
df_wrdc = df_wrdc[df_wrdc.year >= 1970]



In [ ]:
# prepare a new dataframe to create the bar chart for wrd 
df_wrd_calc = pd.DataFrame()
for year in years:
    for religion in religions:
        data = []
        for iso3 in iso3s:
            df_tmp = df_wrdc[(df_wrdc['year'] == year) & (df_wrdc['iso3'] == iso3)]
            if df_tmp.empty:
                data.append(0)
            else:
                data.append(df_wrdc.loc[((df_wrdc['year'] == year) & (df_wrdc['iso3'] == iso3)), religion+'_p'].values[0])
        df_wrd_calc[str(year)+'_'+religion] = data
        

In [ ]:
# draw the wrd data plot
ind = [x for x, _ in enumerate(iso3s)]

for year in years:
# for year in [2000]:
    fig = plt.figure(year, figsize=(14, 10))
    ax = plt.subplot(223)
    
    # draw the bar chart
    plt.bar(ind, df_wrd_calc[str(year)+'_christianity'], width=0.8, label='Christianity', 
            bottom=df_wrd_calc[str(year)+'_islam']+df_wrd_calc[str(year)+'_buddhism']+
            df_wrd_calc[str(year)+'_hindu']+df_wrd_calc[str(year)+'_jews'])
    
    plt.bar(ind, df_wrd_calc[str(year)+'_islam'], width=0.8, label='Islam', 
            bottom=df_wrd_calc[str(year)+'_buddhism']+df_wrd_calc[str(year)+'_hindu']+
            df_wrd_calc[str(year)+'_jews'])
    
    plt.bar(ind, df_wrd_calc[str(year)+'_buddhism'], width=0.8, label='Buddhism', 
            bottom=df_wrd_calc[str(year)+'_hindu']+df_wrd_calc[str(year)+'_jews'])
    
    plt.bar(ind, df_wrd_calc[str(year)+'_hindu'], width=0.8, label='Hindu', 
            bottom=df_wrd_calc[str(year)+'_jews'])
    
    plt.bar(ind, df_wrd_calc[str(year)+'_jews'], width=0.8, label='Jews')
    
    # draw the scatter plot
    i = 0
    for iso3 in iso3s:
        try:
            attack_percentage = df_gtdc.loc[(df_gtdc['iyear'] == year) & (df_gtdc['iso3'] == iso3), 'percentage'].values[0]
        except IndexError:
            attack_percentage = 0
        scatter = plt.scatter(i, attack_percentage, label='Attack in '+iso3, zorder=3)
        i = i + 1
    
    plt.xticks(ind, iso3s)
    plt.ylabel("Population %")
    plt.xlabel("Countries")
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.title(str(year)+" Population by Religion")
    plt.ylim=1.0

    plt.show()